# Chess

In [1]:
# Libraries
import itertools
from importlib import resources
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo
import random
import time

from drdt.helper_functions import DecisionRuleCreatorFromDecisionTable, Reduction, R_SR, R_AD, SAlphaStep, SPlus, SMax, NCover, NGreedy
from drdt.algorithms import DynamicProgrammingAlgorithms, A_C_N, A_C_G
from drdt.helper_functions import DecisionRuleCreatorFromDecisionTable
pd.options.mode.chained_assignment = None


In [2]:
problems = ['AR', 'EAR', 'SR', 'ESR', 'AD', 'EAD']
dataset_name = "Chess"


# Loading Data

In [3]:
with resources.path('datasets.DecisionRuleSystems', 'DRS_chess') as dataset_path:
    S = pd.read_csv(dataset_path).applymap(lambda x: str(x) if pd.notnull(x) else x)
S = S.dropna(axis=1, how='all') # Drop the columns with all None
S

/var/folders/06/0m2hb6f927s6rdyhxjrbss740r2h_1/T/ipykernel_83429/42485626.py:1: DeprecationWarning: path is deprecated. Use files() instead. Refer to https://importlib-resources.readthedocs.io/en/latest/using.html#migrating-from-legacy for migration advice.
  with resources.path('datasets.DecisionRuleSystems', 'DRS_chess') as dataset_path:
/var/folders/06/0m2hb6f927s6rdyhxjrbss740r2h_1/T/ipykernel_83429/42485626.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  S = pd.read_csv(dataset_path).applymap(lambda x: str(x) if pd.notnull(x) else x)


,bkblk,bknwy,bkon8,bkona,bkspr,bkxbq,bkxcr,bkxwp,blxwp,bxqsq,...,skrxp,spcop,stlmt,thrsk,wkcti,wkna8,wknck,wkovl,wkpos,class
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,t,NaN,NaN,NaN,NaN,nowin
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,t,NaN,NaN,NaN,NaN,nowin
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,t,NaN,NaN,NaN,NaN,nowin
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,t,NaN,NaN,NaN,NaN,nowin
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,won
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nowin
3129,t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,won
3130,t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,won
3131,t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,won


# Dataset Analyses

In [4]:
n = len(S.columns)-1
print(f"Number of features  = {n}")

Number of features  = 35


In [5]:
print(f"Does # of features = d  = {not S.dropna().empty}")

Does # of features = d  = False


In [6]:
# Count non-NaN values for each row
non_nan_counts = S.count(axis=1)

# Find the index
max_non_nan_row_index = non_nan_counts.idxmax()

# Retrieve the row
max_non_nan_row = S.loc[max_non_nan_row_index]

# Number of non-NaN values in the row
max_non_nan_count = non_nan_counts[max_non_nan_row_index]

d = max_non_nan_count - 1

print(f"d = {d}") # remove 1 because of last class column

d = 11


# Combinations

In [7]:
column_values = []
for column in S.columns[:-1]:
    unique_values = list(S[column].dropna().unique())
    column_values.append(unique_values)

num_combinations = 1000
combinations = [
    tuple(random.choice(column) for column in column_values)
    for _ in range(num_combinations)
]

In [8]:
column_values_extension = []
for column in S.columns[:-1]:
    unique_values_extension = list(S[column].dropna().unique()) + ['*']
    column_values_extension.append(unique_values_extension)

num_combinations = 1000
extended_combinations = [
    tuple(random.choice(column) for column in column_values_extension)
    for _ in range(num_combinations)
]

# Node Cover

In [9]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NC AR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████| 1000/1000 [01:07<00:00, 14.89it/s]

NC AR
(Max Depth, Min Depth, Average Depth, Running time) = (35, 33, 34.875, 67.18345189094543)


In [10]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NC EAR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████| 1000/1000 [00:59<00:00, 16.86it/s]

NC EAR
(Max Depth, Min Depth, Average Depth, Running time) = (35, 32, 34.597, 59.30390810966492)


In [11]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="SR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NC SR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████| 1000/1000 [27:53<00:00,  1.67s/it]

NC SR
(Max Depth, Min Depth, Average Depth, Running time) = (30, 11, 16.321, 1673.5979969501495)


In [12]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="ESR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NC ESR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████| 1000/1000 [27:56<00:00,  1.68s/it]

NC ESR
(Max Depth, Min Depth, Average Depth, Running time) = (31, 11, 20.987, 1676.5891561508179)


In [13]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AD", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NC AD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████| 1000/1000 [28:55<00:00,  1.74s/it]

NC AD
(Max Depth, Min Depth, Average Depth, Running time) = (32, 11, 21.413, 1735.410898923874)


In [14]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAD", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NC EAD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████| 1000/1000 [28:52<00:00,  1.73s/it]

NC EAD
(Max Depth, Min Depth, Average Depth, Running time) = (32, 11, 24.942, 1732.6853840351105)


# Node Cover Greedy

In [15]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NCgreedy AR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████| 1000/1000 [02:10<00:00,  7.65it/s]

NCgreedy AR
(Max Depth, Min Depth, Average Depth, Running time) = (35, 32, 34.787, 130.67185378074646)


In [16]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NCgreedy EAR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████| 1000/1000 [02:00<00:00,  8.28it/s]

NCgreedy EAR
(Max Depth, Min Depth, Average Depth, Running time) = (35, 28, 33.909, 120.8356580734253)


In [17]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="SR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NCgreedy SR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████| 1000/1000 [30:25<00:00,  1.83s/it]

NCgreedy SR
(Max Depth, Min Depth, Average Depth, Running time) = (32, 5, 16.563, 1825.5230588912964)


In [18]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="ESR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NCgreedy ESR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████| 1000/1000 [30:24<00:00,  1.82s/it]

NCgreedy ESR
(Max Depth, Min Depth, Average Depth, Running time) = (31, 5, 20.601, 1824.246069908142)


In [19]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AD", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NCgreedy AD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████| 1000/1000 [31:56<00:00,  1.92s/it]

NCgreedy AD
(Max Depth, Min Depth, Average Depth, Running time) = (31, 10, 21.68, 1916.9131581783295)


In [20]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAD", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NCgreedy EAD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))


100%|███████████████████████████████████████| 1000/1000 [32:22<00:00,  1.94s/it]

NCgreedy EAD
(Max Depth, Min Depth, Average Depth, Running time) = (32, 9, 23.495, 1942.417209148407)


# Greedy

In [21]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="AR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("Greedy AR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████| 1000/1000 [01:04<00:00, 15.50it/s]

Greedy AR
(Max Depth, Min Depth, Average Depth, Running time) = (35, 32, 34.734, 64.53368997573853)


In [22]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="EAR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("Greedy EAR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████| 1000/1000 [00:46<00:00, 21.70it/s]

Greedy EAR
(Max Depth, Min Depth, Average Depth, Running time) = (35, 27, 33.189, 46.08693885803223)


In [23]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="SR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("Greedy SR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████| 1000/1000 [30:26<00:00,  1.83s/it]

Greedy SR
(Max Depth, Min Depth, Average Depth, Running time) = (21, 3, 11.031, 1826.229479789734)


In [24]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="ESR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("Greedy ESR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████| 1000/1000 [29:11<00:00,  1.75s/it]

Greedy ESR
(Max Depth, Min Depth, Average Depth, Running time) = (27, 3, 14.771, 1751.692036151886)


In [25]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="AD")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("Greedy AD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████| 1000/1000 [30:23<00:00,  1.82s/it]

Greedy AD
(Max Depth, Min Depth, Average Depth, Running time) = (24, 8, 14.999, 1823.672749042511)


In [26]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="EAD")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("Greedy EAD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████| 1000/1000 [30:06<00:00,  1.81s/it]

Greedy EAD
(Max Depth, Min Depth, Average Depth, Running time) = (27, 9, 18.012, 1806.9855279922485)


# Molecular Biology

In [27]:
# Libraries
import itertools
from importlib import resources
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo
import random
import time

from drdt.helper_functions import DecisionRuleCreatorFromDecisionTable, Reduction, R_SR, R_AD, SAlphaStep, SPlus, SMax, NCover, NGreedy
from drdt.algorithms import DynamicProgrammingAlgorithms, A_C_N, A_C_G
from drdt.helper_functions import DecisionRuleCreatorFromDecisionTable
pd.options.mode.chained_assignment = None


In [28]:
problems = ['AR', 'EAR', 'SR', 'ESR', 'AD', 'EAD']
dataset_name = "MolecularBiology"


# Loading Data

In [29]:
with resources.path('datasets.DecisionRuleSystems', 'DRS_molecular_biology') as dataset_path:
    S = pd.read_csv(dataset_path).applymap(lambda x: str(x) if pd.notnull(x) else x)
S = S.dropna(axis=1, how='all') # Drop the columns with all None
S

/var/folders/06/0m2hb6f927s6rdyhxjrbss740r2h_1/T/ipykernel_83429/4126655002.py:1: DeprecationWarning: path is deprecated. Use files() instead. Refer to https://importlib-resources.readthedocs.io/en/latest/using.html#migrating-from-legacy for migration advice.
  with resources.path('datasets.DecisionRuleSystems', 'DRS_molecular_biology') as dataset_path:
/var/folders/06/0m2hb6f927s6rdyhxjrbss740r2h_1/T/ipykernel_83429/4126655002.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  S = pd.read_csv(dataset_path).applymap(lambda x: str(x) if pd.notnull(x) else x)


,Base1,Base2,Base3,Base4,Base5,Base6,Base7,Base8,Base9,Base10,...,Base52,Base53,Base54,Base55,Base56,Base57,Base58,Base59,Base60,class
0,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IE
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
3001,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
3002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN,N
3003,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N


# Dataset Analyses

In [30]:
n = len(S.columns)-1
print(f"Number of features  = {n}")

Number of features  = 60


In [31]:
print(f"Does # of afeatures = d  = {not S.dropna().empty}")

Does # of afeatures = d  = False


In [32]:
# Count non-NaN values for each row
non_nan_counts = S.count(axis=1)

# Find the index
max_non_nan_row_index = non_nan_counts.idxmax()

# Retrieve the row
max_non_nan_row = S.loc[max_non_nan_row_index]

# Number of non-NaN values in the row
max_non_nan_count = non_nan_counts[max_non_nan_row_index]

d = max_non_nan_count - 1

print(f"d = {d}") # remove 1 because of last class column

d = 5


# Combinations

In [33]:
column_values = []
for column in S.columns[:-1]:
    unique_values = list(S[column].dropna().unique())
    column_values.append(unique_values)

num_combinations = 1000
combinations = [
    tuple(random.choice(column) for column in column_values)
    for _ in range(num_combinations)
]

In [34]:
column_values_extension = []
for column in S.columns[:-1]:
    unique_values_extension = list(S[column].dropna().unique()) + ['*']
    column_values_extension.append(unique_values_extension)

num_combinations = 1000
extended_combinations = [
    tuple(random.choice(column) for column in column_values_extension)
    for _ in range(num_combinations)
]

# Node Cover

In [35]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NC AR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████| 1000/1000 [01:18<00:00, 12.67it/s]

NC AR
(Max Depth, Min Depth, Average Depth, Running time) = (60, 52, 57.396, 78.95450210571289)


In [36]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NC EAR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████| 1000/1000 [01:16<00:00, 13.10it/s]

NC EAR
(Max Depth, Min Depth, Average Depth, Running time) = (60, 51, 56.518, 76.32157707214355)


In [37]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="SR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NC SR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|█████████████████████████████████████| 1000/1000 [1:36:56<00:00,  5.82s/it]

NC SR
(Max Depth, Min Depth, Average Depth, Running time) = (42, 10, 14.636, 5816.026385068893)


In [38]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="ESR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NC ESR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|█████████████████████████████████████| 1000/1000 [1:37:05<00:00,  5.83s/it]

NC ESR
(Max Depth, Min Depth, Average Depth, Running time) = (51, 10, 18.401, 5825.458536863327)


In [39]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AD", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NC AD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|█████████████████████████████████████| 1000/1000 [1:39:54<00:00,  5.99s/it]

NC AD
(Max Depth, Min Depth, Average Depth, Running time) = (56, 10, 36.28, 5994.165041923523)


In [40]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAD", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NC EAD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|█████████████████████████████████████| 1000/1000 [1:39:55<00:00,  6.00s/it]

NC EAD
(Max Depth, Min Depth, Average Depth, Running time) = (56, 10, 40.711, 5995.965836048126)


# Node Cover Greedy

In [41]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NCgreedy AR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████| 1000/1000 [03:59<00:00,  4.18it/s]

NCgreedy AR
(Max Depth, Min Depth, Average Depth, Running time) = (60, 47, 54.963, 239.42307806015015)


In [42]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NCgreedy EAR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████| 1000/1000 [03:50<00:00,  4.33it/s]

NCgreedy EAR
(Max Depth, Min Depth, Average Depth, Running time) = (60, 45, 53.187, 230.98719906806946)


In [43]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="SR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NCgreedy SR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|█████████████████████████████████████| 1000/1000 [1:42:30<00:00,  6.15s/it]

NCgreedy SR
(Max Depth, Min Depth, Average Depth, Running time) = (55, 2, 12.881, 6150.352170944214)


In [44]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="ESR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NCgreedy ESR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|█████████████████████████████████████| 1000/1000 [1:43:29<00:00,  6.21s/it]

NCgreedy ESR
(Max Depth, Min Depth, Average Depth, Running time) = (55, 2, 16.109, 6209.20591211319)


In [45]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AD", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NCgreedy AD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|█████████████████████████████████████| 1000/1000 [1:47:09<00:00,  6.43s/it]

NCgreedy AD
(Max Depth, Min Depth, Average Depth, Running time) = (55, 13, 32.85, 6429.634438991547)


In [46]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAD", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NCgreedy EAD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))


100%|█████████████████████████████████████| 1000/1000 [1:46:43<00:00,  6.40s/it]

NCgreedy EAD
(Max Depth, Min Depth, Average Depth, Running time) = (55, 13, 36.211, 6403.843142032623)


# Greedy

In [47]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="AR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("Greedy AR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████| 1000/1000 [01:02<00:00, 15.91it/s]

Greedy AR
(Max Depth, Min Depth, Average Depth, Running time) = (60, 45, 53.54, 62.85799312591553)


In [48]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="EAR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("Greedy EAR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████| 1000/1000 [00:56<00:00, 17.77it/s]

Greedy EAR
(Max Depth, Min Depth, Average Depth, Running time) = (59, 40, 51.265, 56.260916233062744)


In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="SR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("Greedy SR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

 49%|██████████████████▌                   | 488/1000 [57:48<1:00:07,  7.05s/it]

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="ESR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("Greedy ESR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="AD")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("Greedy AD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="EAD")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("Greedy EAD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

# Mushroom

In [ ]:
# Libraries
import itertools
from importlib import resources
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo
import random
import time

from drdt.helper_functions import DecisionRuleCreatorFromDecisionTable, Reduction, R_SR, R_AD, SAlphaStep, SPlus, SMax, NCover, NGreedy
from drdt.algorithms import DynamicProgrammingAlgorithms, A_C_N, A_C_G
from drdt.helper_functions import DecisionRuleCreatorFromDecisionTable
pd.options.mode.chained_assignment = None


In [ ]:
problems = ['AR', 'EAR', 'SR', 'ESR', 'AD', 'EAD']
dataset_name = "Mushroom"


In [ ]:
with resources.path('datasets.DecisionRuleSystems', 'DRS_mushroom') as dataset_path:
    S = pd.read_csv(dataset_path).applymap(lambda x: str(x) if pd.notnull(x) else x)
S = S.dropna(axis=1, how='all') # Drop the columns with all None
S

# Dataset Analyses

In [ ]:
n = len(S.columns)-1
print(f"Number of features  = {n}")

In [ ]:
print(f"Does # of afeatures = d  = {not S.dropna().empty}")

In [ ]:
# Count non-NaN values for each row
non_nan_counts = S.count(axis=1)

# Find the index
max_non_nan_row_index = non_nan_counts.idxmax()

# Retrieve the row
max_non_nan_row = S.loc[max_non_nan_row_index]

# Number of non-NaN values in the row
max_non_nan_count = non_nan_counts[max_non_nan_row_index]

d = max_non_nan_count - 1

print(f"d = {d}") # remove 1 because of last class column

# Combinations

In [ ]:
column_values = []
for column in S.columns[:-1]:
    unique_values = list(S[column].dropna().unique())
    column_values.append(unique_values)

num_combinations = 1000
combinations = [
    tuple(random.choice(column) for column in column_values)
    for _ in range(num_combinations)
]

In [ ]:
column_values_extension = []
for column in S.columns[:-1]:
    unique_values_extension = list(S[column].dropna().unique()) + ['*']
    column_values_extension.append(unique_values_extension)

num_combinations = 1000
extended_combinations = [
    tuple(random.choice(column) for column in column_values_extension)
    for _ in range(num_combinations)
]

# Node Cover

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NC AR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NC EAR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="SR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NC SR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="ESR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NC ESR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AD", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NC AD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAD", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NC EAD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

# Node Cover Greedy

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NCgreedy AR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NCgreedy EAR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="SR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NCgreedy SR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="ESR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NCgreedy ESR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AD", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NCgreedy AD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAD", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NCgreedy EAD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))


# Greedy

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="AR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("Greedy AR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="EAR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("Greedy EAR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="SR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("Greedy SR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="ESR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("Greedy ESR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="AD")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("Greedy AD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="EAD")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("Greedy EAD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

# Soybean

In [ ]:
# Libraries
import itertools
from importlib import resources
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo
import random
import time

from drdt.helper_functions import DecisionRuleCreatorFromDecisionTable, Reduction, R_SR, R_AD, SAlphaStep, SPlus, SMax, NCover, NGreedy
from drdt.algorithms import DynamicProgrammingAlgorithms, A_C_N, A_C_G
from drdt.helper_functions import DecisionRuleCreatorFromDecisionTable
pd.options.mode.chained_assignment = None


In [ ]:
problems = ['AR', 'EAR', 'SR', 'ESR', 'AD', 'EAD']
dataset_name = "Soybean"


In [ ]:
with resources.path('datasets.DecisionRuleSystems', 'DRS_soybean') as dataset_path:
    S = pd.read_csv(dataset_path).applymap(lambda x: str(x) if pd.notnull(x) else x)
S = S.dropna(axis=1, how='all') # Drop the columns with all None
S

# Dataset Analyses

In [ ]:
n = len(S.columns)-1
print(f"Number of features  = {n}")

In [ ]:
print(f"Does # of afeatures = d  = {not S.dropna().empty}")

In [ ]:
# Count non-NaN values for each row
non_nan_counts = S.count(axis=1)

# Find the index
max_non_nan_row_index = non_nan_counts.idxmax()

# Retrieve the row
max_non_nan_row = S.loc[max_non_nan_row_index]

# Number of non-NaN values in the row
max_non_nan_count = non_nan_counts[max_non_nan_row_index]

d = max_non_nan_count - 1

print(f"d = {d}") # remove 1 because of last class column

# Combinations

In [ ]:
column_values = []
for column in S.columns[:-1]:
    unique_values = list(S[column].dropna().unique())
    column_values.append(unique_values)

num_combinations = 1000
combinations = [
    tuple(random.choice(column) for column in column_values)
    for _ in range(num_combinations)
]

In [ ]:
column_values_extension = []
for column in S.columns[:-1]:
    unique_values_extension = list(S[column].dropna().unique()) + ['*']
    column_values_extension.append(unique_values_extension)

num_combinations = 1000
extended_combinations = [
    tuple(random.choice(column) for column in column_values_extension)
    for _ in range(num_combinations)
]

# Node Cover

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NC AR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NC EAR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="SR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NC SR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="ESR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NC ESR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AD", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NC AD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAD", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NC EAD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

# Node Cover Greedy

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NCgreedy AR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NCgreedy EAR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="SR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NCgreedy SR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="ESR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NCgreedy ESR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AD", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NCgreedy AD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAD", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("NCgreedy EAD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))


# Greedy

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="AR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("Greedy AR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="EAR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("Greedy EAR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="SR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("Greedy SR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="ESR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("Greedy ESR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="AD")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("Greedy AD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="EAD")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
end_time = time.time()
    
print("Greedy EAD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))